# Question

## 5 - Pengujian Hipotesis

1. Apakah terdapat pengaruh besar pada deposit_type dengan pembatalan hotel?
  - H0 : Tidak terdapat pengaruh besar anatara deposit_type dengan pembatalan hotel
  - H1 : Terdapat pengaruh besar antara deposit_type dengan pembatalan hotel
  
2. Apakah terdapat perbedaan siginifikan antara rerata ADR untuk pemesanan yang dibatalkan dan tidak?
  - H0 : Tidak terdapat perbedaan signifikan antara rata-rata ADR antara reservasi yang dibatalkan dan tidak dibatalkan
  - H1 : Terdapat perbedaan signifikan dalam rata-rata ADR antara reservasi yang dibatalkan dan tidak dibatalkan

3. Apakah variansadr city hotel dengan resort hotel sama?
  - H0 : Varians adr city hotel dengan resort hotel berbeda
  - H1 : Varians adr city hotel dengan resort hotel sama

4. Apakah terdapat perbedaan signifikan dalam proporsi pembatalan antara pemesanan melalui distribusi agensi atau organizer dengan yang langsung (GDS, direct, corporate)?
  - H0 : Tidak terdapat perbedaan signifikan dalam proporsi pembatalan antara agensi dan organizer dengan lainnya
  - H1 : Terdapat perbedaan signifikan dalam proporsi pembatalan antara agensi dan organizer dengan lainnya

# Import Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency, f, norm, normaltest, mannwhitneyu
from statsmodels.stats.proportion import proportions_ztest

import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
sns.set_style("darkgrid")

# Import Dataset

In [2]:
df = pd.read_csv('hotel_bookings.csv')

df.head(5)

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,4,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Direct,Direct,0,0,0,A,C,0,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Corporate,Corporate,0,0,0,A,A,0,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,0.0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03


In [3]:
print(df.columns)

Index(['hotel', 'is_canceled', 'lead_time', 'arrival_date_year',
       'arrival_date_month', 'arrival_date_week_number',
       'arrival_date_day_of_month', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'adults', 'children', 'babies', 'meal',
       'country', 'market_segment', 'distribution_channel',
       'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'reserved_room_type',
       'assigned_room_type', 'booking_changes', 'deposit_type', 'agent',
       'company', 'days_in_waiting_list', 'customer_type', 'adr',
       'required_car_parking_spaces', 'total_of_special_requests',
       'reservation_status', 'reservation_status_date'],
      dtype='object')


# Analysis

## 1. Apakah terdapat pengaruh besar pada deposit_type dengan pembatalan hotel?
- H0 : Tidak terdapat pengaruh besar anatara deposit_type dengan pembatalan hotel
- H1 : Terdapat pengaruh besar antara deposit_type dengan pembatalan hotel

In [8]:
# process data
data = df[['is_canceled', 'deposit_type']]
cross_tab = pd.crosstab(data['is_canceled'], data['deposit_type'])

# uji chi-square
stat, p_value, dof, expected = chi2_contingency(cross_tab)
print('p-value : ', p_value)

p-value :  0.0


Nilai p-value kurang dari 0.05 sehingga H0 ditolak. Menunjukkan terdapat korelasi atau pengaruh besar antara deposit_type dengan pembatalan pemesanan hotel, sehingga deposit_type bisa dijadikan parameter untuk menghindari pembatalan hotel.

## 2. Apakah terdapat perbedaan siginifikan antara rerata ADR untuk pemesanan yang dibatalkan dan tidak?
  - H0 : Tidak terdapat perbedaan signifikan antara rata-rata ADR antara reservasi yang dibatalkan dan tidak dibatalkan
  - H1 : Terdapat perbedaan signifikan dalam rata-rata ADR antara reservasi yang dibatalkan dan tidak dibatalkan

In [5]:
data = df[(df['adr']<5000) & (df['adr']>0)]

# separate adr data for canceled and not canceled bookings
adr_canceled = data.loc[data['is_canceled'] == 1, 'adr']
adr_not_canceled = data.loc[data['is_canceled'] == 0, 'adr']

# not normal dist using mannwhitneyu
stat, p_value = mannwhitneyu(adr_canceled, adr_not_canceled)
print('p-value : ', p_value)

p-value :  5.4196022607713324e-36


Nilai p-value kurang dari 0.05 sehingga H0 ditolak. Menunjukkan terdapat perbedaan signifikan antara rata-rata ADR antara reservasi yang dibatalkan dan tidak dibatalkan.

## 3. Apakah varians adr city hotel dengan resort hotel sama?
  - H0 : Varians adr city hotel dengan resort hotel berbeda
  - H1 : Varians adr city hotel dengan resort hotel sama

In [6]:
# filter hanya city hotel dan resort hotel
city_hotel = df[df['hotel'] == 'City Hotel']['adr']
resort_hotel = df[df['hotel'] == 'Resort Hotel']['adr']

# uji f
F = city_hotel.var() / resort_hotel.var()
DF1 = len(city_hotel) - 1
DF2 = len(resort_hotel) - 1
critical_value = f.sf(F, DF1, DF2)

print('F value : ', F)
print('F critical value : ', critical_value)

F value :  0.5036110208183524
F critical value :  0.9999999999999999


Nilai F-kritis lebih besar dibandingkan dengan rasio varians atau nilai F. segingga menolak H0 dan menerima H1 yang dapat dismpulkan jika varians adr pada city hotel sama dengan varians pada resort hotel.

## 4. Apakah terdapat perbedaan signifikan dalam proporsi pembatalan antara pemesanan melalui distribusi agensi atau organizer dengan yang langsung (GDS, direct, corporate)?
- H0 : Tidak terdapat perbedaan signifikan dalam proporsi pembatalan antara agensi dan organizer dengan lainnya
- H1 : Terdapat perbedaan signifikan dalam proporsi pembatalan antara agensi dan organizer dengan lainnya

In [7]:
# define data
ta_to = df[df['distribution_channel'] == 'TA/TO']
other = df[df['distribution_channel'].isin(['Corporate', 'Direct', 'GDS'])]

# cancel each data
n_ta_to_cancel = ta_to['is_canceled'].sum()
n_other_cancel = other['is_canceled'].sum()

# count data
n_ta_to = len(ta_to)
n_other = len(other)

# proprotion cancel
p_ta_to = n_ta_to_cancel / n_ta_to
p_other = n_other_cancel / n_other

# diff proportion
p_diff = p_ta_to - p_other

# hypothesting test
z_score, p_value = proportions_ztest([n_ta_to_cancel, n_other_cancel], [n_ta_to, n_other])

print('p-value : ', p_value)

p-value :  0.0


Nilai p-value kurang dari 0.05 sehingga H0 ditolak. Menunjukkan terdapat perbedaan signifikan proporsi pembatalan pada pemesanan melalui agensi atau organizer dibandingkan dengan non agensi dan non organizer. Hal ini dapat menjadi pertimbangan pada pihak hotel untuk menekan proporsi pembatalan pada distribusi agensi dan organizer seperti dengan memberikan diskon dan yang lainnya.